In [5]:
import os
os.environ['SIRIUS_URL_CONSTS'] = 'http://127.0.0.1'

In [6]:
%gui qt5
%matplotlib qt5

from functools import partial as _partial
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
import matplotlib.gridspec as mpl_gs
import matplotlib.cm as cmap
import matplotlib.colors as mpl_colors
from matplotlib import rcParams
rcParams.update(
    {'font.size': 16,'lines.linewidth': 2, 'axes.grid': True})

import pyaccel
import mathphys
from pymodels import si
from apsuite.dynap import DynapXY
from apsuite.optics_analysis import TuneCorr

In [7]:
model = si.create_accelerator()
model.cavity_on = False
model.radiation_on = False
dynap = DynapXY(model)

In [18]:
tunecorr = TuneCorr(model, 'SI')

In [19]:
tunecorr.correct_parameters([49.186188, 14.2300])

1

In [20]:
twi, _ = pyaccel.optics.calc_twiss(dynap._acc)
twi.mux[-1]/2/np.pi, twi.muy[-1]/2/np.pi

(49.18574477838651, 14.230530180928666)

In [8]:
dynap.params.turn_by_turn = True
dynap.params.x_nrpts = 70
dynap.params.y_nrpts = 30
dynap.params.x_min = -0.012
dynap.params.x_max = 0.00
dynap.params.y_min = 0
dynap.params.y_max = 0.004
print(dynap)

nrturns      : 512
turn_by_turn : True
x_nrpts      : 70
y_nrpts      : 30
x_min [m]    : -0.012
x_max [m]    : 0
y_min [m]    : 0
y_max [m]    : 0.004
de_offset    : 0
xl_off [rad] : 1e-05
yl_off [rad] : 1e-05
intnux       : 49.00 (for graphs)
intnuy       : 14.00 (for graphs)



In [17]:
dynap.do_tracking()

In [18]:
dynap.process_data()

In [24]:
# dynap.save_data('/home/fernando/delta_tune2.pickle')
dynap.load_and_apply('/home/fernando/delta_tune2.pickle')
dynap.process_data()

In [9]:
dynap.load_and_apply('/home/fernando/nominal.pickle')
dynap.process_data()

# Diffusion Plot

In [10]:
bounds = [
    dynap.params.intnux, dynap.params.intnux + 0.5,
    dynap.params.intnuy, dynap.params.intnuy + 0.5]
orders = 6
symmetry = 1
resons = dynap.calc_resonances_for_bounds(bounds, orders=orders, symmetry=symmetry)

In [11]:
map2xy = dynap.map_resons2real_plane(resons, maxdist=2e-3, min_diffusion=1e-5)
numb = [x.size for x in map2xy]
filter_resons = []
for i, (res, nu) in enumerate(zip(resons, numb)):
    if nu > 15:
        print(f'{i:2}: {res}  {nu:3}')
        filter_resons.append(res)

to_remove = [-2]
for idx in sorted(to_remove, reverse=True):
    del filter_resons[idx]

print()
for i, reson in enumerate(filter_resons):
    print(f'{reson}')

15: (4, 2, 225)   20
19: (2, 4, 155)   17
38: (0, 4, 57)   22
41: (0, 6, 85)   47
52: (-4, 1, -182)   60
63: (3, 3, 190)   17
70: (-2, 1, -84)   35
90: (1, 2, 78)   27

(4, 2, 225)
(2, 4, 155)
(0, 4, 57)
(0, 6, 85)
(-4, 1, -182)
(3, 3, 190)
(1, 2, 78)


In [12]:
f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=5e-3, min_diffusion=1e-5)
# f, ax, ay = dynap.make_figure_diffusion(contour=True, resons=filter_resons, maxdist=1e-3)
f.show()
# f = dynap.make_figure_diffusion(contour=False)

# X and Y map to tune plot

In [16]:
f, axx, ayy = dynap.make_figure_map_real2tune_planes(resons)

In [13]:
ax.plot(dynap.x_dynap*1e3, dynap.y_dynap*1e3, '-xk')
# ax.plot(dynap.x_dynap[tup]*1e3, dynap.y_dynap[tup]*1e3, '-xr')
f.canvas.draw()

In [14]:
dynap.y_dynap

array([0.00303448, 0.00137931, 0.00206897, 0.00303448, 0.00165517,
       0.00137931, 0.00234483, 0.00262069, 0.00206897, 0.00027586,
       0.00165517, 0.00151724, 0.00372414, 0.00344828, 0.00068966,
       0.00165517, 0.00358621, 0.00137931, 0.00193103, 0.00110345,
       0.0017931 , 0.00124138, 0.004     , 0.00206897, 0.        ,
       0.00041379, 0.00303448, 0.0017931 , 0.00110345, 0.00248276,
       0.00137931, 0.00331034, 0.00055172, 0.00317241, 0.00013793,
       0.00096552, 0.0017931 , 0.00248276, 0.00151724, 0.00124138,
       0.00344828, 0.00206897, 0.00082759, 0.00289655, 0.00275862,
       0.00165517, 0.00331034, 0.00193103, 0.00289655, 0.0017931 ,
       0.00151724, 0.00124138, 0.00358621, 0.00386207, 0.00193103,
       0.0022069 , 0.00317241, 0.00206897, 0.00289655])

In [33]:
x, y = dynap.y_dynap, dynap.x_dynap


r_sqr = x*x + y*y
theta = np.arctan2(y, x)
tup = np.lexsort((-theta, ))



In [11]:
for x, y in zip(dynap.x_dynap, dynap.y_dynap):
    print(f'{(x,y)}')

(0.0, 0.0)
(0.0, 0.00013793103448275863)
(0.0, 0.00027586206896551725)
(0.0, 0.00041379310344827585)
(0.0, 0.0005517241379310345)
(0.0, 0.0006896551724137932)
(0.0, 0.0008275862068965517)
(0.0, 0.0009655172413793104)
(0.0, 0.001103448275862069)
(0.0, 0.0012413793103448277)
(0.0, 0.0013793103448275863)
(0.0, 0.001517241379310345)
(0.0, 0.0016551724137931034)
(0.0, 0.001793103448275862)
(0.0, 0.0019310344827586207)
(0.0, 0.0020689655172413794)
(0.0, 0.002206896551724138)
(0.0, 0.0023448275862068967)
(0.0, 0.0024827586206896553)
(0.0, 0.002620689655172414)
(0.0, 0.0027586206896551726)
(0.0, 0.0028965517241379313)
(0.0, 0.00303448275862069)
(0.0, 0.0031724137931034486)
(0.0, 0.003310344827586207)
(0.0, 0.0034482758620689655)
(0.0, 0.003586206896551724)
(0.0, 0.0037241379310344828)
(0.0, 0.0038620689655172414)
(0.0, 0.004)
(-0.00017391304347826042, 0.004)
(-0.00034782608695652084, 0.004)
(-0.0005217391304347813, 0.004)
(-0.0006956521739130434, 0.004)
(-0.0008695652173913038, 0.004)
(-0.0010